In [1]:
from __future__ import print_function

import numpy as np

from sklearn import __version__ as sklearn_version
print('Sklearn version:', sklearn_version)

Sklearn version: 0.19.1


# The data

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.


In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'),
                 categories=categories, shuffle=True, random_state=42)



## Build a pipeline

In [3]:
#Define the pipeline

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB


text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

# Fit all the pipeline
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=5000, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
       ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [4]:
#Evaluate test data
twenty_test = fetch_20newsgroups(subset='test',
                    remove=('headers', 'footers', 'quotes'),
                    categories=categories, 
                    shuffle=True, random_state=42)

predicted = text_clf.predict(twenty_test.data)

print('Test accuracy:', np.mean(predicted == twenty_test.target))

Test accuracy: 0.7989347536617842


## Change classifier in the pipeline
    - LinearSVC
    - k-NN
    - Random forest

In [5]:
from sklearn.svm import LinearSVC

text_clf_svm = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                    ])

#Fit
_ = text_clf_svm.fit(twenty_train.data, twenty_train.target)

# Predict
predicted = text_clf_svm.predict(twenty_test.data)

# Evaluate accuracy
print('Test accuracy:', np.mean(predicted == twenty_test.target))        

Test accuracy: 0.8089214380825566


In [6]:
from sklearn.neighbors import KNeighborsClassifier

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier(n_neighbors=20)),
                    ])

_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

Test accuracy: 0.2529960053262317


In [7]:
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(n_estimators=100)),
                    ])

_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

Test accuracy: 0.7430093209054593


## Use features from a factorization instead the provided by the tf-idf

In [8]:
# Text preprocessing, tokenizing and filtering of stopwords
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=5000,
                                stop_words='english')
X_train_counts = tf_vectorizer.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 5000)

In [11]:
%%time

from sklearn.decomposition import  LatentDirichletAllocation

n_components = 6
n_top_words = 20

lda = LatentDirichletAllocation(n_components=n_components,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(X_train_counts)


CPU times: user 4 s, sys: 48.8 ms, total: 4.05 s
Wall time: 4.12 s


In [13]:
lda.transform(X_train_counts).shape

(2257, 6)

In [10]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
church pope catholic marriage authority married orthodox canon schism mass liturgy bishop ceremony st churches catholics does priest jurisdiction coptic
Topic #1:
image file jpeg use program files images gif color know format does thanks graphics software using version bit available like
Topic #2:
edu com graphics mail send pub keyboard ftp data computer information cs systems software ca faq available gov contact pc
Topic #3:
god people think don jesus just does believe know say like time bible way things good true life christian question
Topic #4:
health use medical years people disease food msg new patients like don doctor research time 1993 10 day know just
Topic #5:
banks gordon skepticism edu soon pitt geb intellect chastity n3jxp dsl shameful cadre surrender father spirit son holy int col



## Pipeline with factorization

In [15]:
%%time

from sklearn.neighbors import KNeighborsClassifier

text_lda_knn = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=10000, stop_words='english')),
                         ('lda', LatentDirichletAllocation(n_components=150, max_iter=15,
                                 learning_method='online',
                                 learning_offset=200.,
                                 random_state=0)),
                         ('clf', KNeighborsClassifier(n_neighbors=10))
                        ])

                         
_ = text_lda_knn.fit(twenty_train.data, twenty_train.target)
predicted = text_lda_knn.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

Test accuracy: 0.7003994673768309
CPU times: user 46.9 s, sys: 3.74 s, total: 50.6 s
Wall time: 50.7 s


In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

text_lda_rf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=10000, stop_words='english')),
                         ('lda', LatentDirichletAllocation(n_components=150, max_iter=15,
                                 learning_method='online',
                                 learning_offset=200.,
                                 random_state=0)),
                         ('clf', RandomForestClassifier(n_estimators=100)),
                        ])

                         
_ = text_lda_rf.fit(twenty_train.data, twenty_train.target)

predicted = text_lda_rf.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

Test accuracy: 0.6970705725699068
CPU times: user 47.4 s, sys: 3.68 s, total: 51 s
Wall time: 51 s


## Optimize a pipeline

In [20]:
%%time

from sklearn.model_selection import RandomizedSearchCV

# Define estimator. No parameters of the search
clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2)),
                ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC()),
                ])

# Specify parameters and distributions to sample from
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_dist = {"vect__max_features": [1000, 2500, 5000, 7500, 10000, None], 
              "vect__stop_words": ['english', None], 
              "clf__C": [.1, .5, 1., 1.5, 2.]}

# Define randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search, return_train_score=True)

# Run the randomized search
random_search.fit(twenty_train.data, twenty_train.target)

print("Done!")

Done!
CPU times: user 19.7 s, sys: 323 ms, total: 20 s
Wall time: 20.2 s


In [21]:
# Load dictionary of search results to a Pandas dataframe
import pandas as pd

df_cv_results = pd.DataFrame.from_dict(random_search.cv_results_)
df_cv_results

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_clf__C,param_vect__max_features,param_vect__stop_words,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.292985,0.105646,0.827647,0.982056,1,2500,english,"{'vect__stop_words': 'english', 'vect__max_fea...",8,0.831341,0.982713,0.820717,0.982713,0.830892,0.980744,0.007994,0.006067,0.004907,0.000928
1,0.297604,0.119258,0.825875,0.944619,0.1,5000,None,"{'vect__stop_words': None, 'vect__max_features...",9,0.832669,0.948803,0.819389,0.945479,0.825566,0.939575,0.004898,0.007005,0.005428,0.003816
2,0.282085,0.108363,0.834293,0.978734,0.5,2500,english,"{'vect__stop_words': 'english', 'vect__max_fea...",7,0.841965,0.978723,0.828685,0.980718,0.832224,0.976760,0.008137,0.005569,0.005617,0.001616
3,0.299318,0.111629,0.856004,0.981392,0.5,5000,english,"{'vect__stop_words': 'english', 'vect__max_fea...",6,0.867198,0.980718,0.847278,0.983378,0.853529,0.980080,0.016055,0.006265,0.008322,0.001428
4,0.320039,0.114126,0.783341,0.978954,2,1000,None,"{'vect__stop_words': None, 'vect__max_features...",10,0.796813,0.977394,0.792829,0.980718,0.760320,0.978752,0.006847,0.006193,0.016338,0.001365
5,0.334155,0.122563,0.861320,0.982499,1.5,None,None,"{'vect__stop_words': None, 'vect__max_features...",4,0.869854,0.982048,0.851262,0.984043,0.862850,0.981408,0.000979,0.006169,0.007670,0.001122
6,0.294638,0.112898,0.868852,0.982500,0.5,None,english,"{'vect__stop_words': 'english', 'vect__max_fea...",1,0.883134,0.983378,0.856574,0.984043,0.866844,0.980080,0.007096,0.007421,0.010940,0.001733
7,0.310869,0.121988,0.866194,0.983386,1.5,10000,english,"{'vect__stop_words': 'english', 'vect__max_fea...",2,0.876494,0.984043,0.853918,0.984043,0.868176,0.982072,0.014606,0.013349,0.009326,0.000929
8,0.331829,0.122588,0.865308,0.982499,1,10000,None,"{'vect__stop_words': None, 'vect__max_features...",3,0.875166,0.982048,0.853918,0.984043,0.866844,0.981408,0.009305,0.007172,0.008746,0.001122
9,0.312945,0.123937,0.860434,0.981170,0.5,None,None,"{'vect__stop_words': None, 'vect__max_features...",5,0.868526,0.980053,0.856574,0.983378,0.856192,0.980080,0.004496,0.005430,0.005728,0.001561


In [19]:
print('Best params:', random_search.best_params_)

Best params: {'vect__stop_words': 'english', 'vect__max_features': None, 'clf__C': 1.5}


In [22]:
# Score & evaluate test data using the best estimator

predicted = random_search.best_estimator_.predict(twenty_test.data)

print('Test accuracy:', np.mean(predicted == twenty_test.target))        

Test accuracy: 0.8249001331557922


## Aditional metrics for multiclass classification

In [23]:
from sklearn import metrics

print(metrics.classification_report(twenty_test.target, 
                                    predicted,
                                    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.78      0.60      0.67       319
         comp.graphics       0.90      0.91      0.91       389
               sci.med       0.81      0.90      0.86       396
soc.religion.christian       0.80      0.84      0.82       398

           avg / total       0.82      0.82      0.82      1502



In [24]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[190,  12,  40,  77],
       [ 12, 355,  21,   1],
       [ 13,  17, 358,   8],
       [ 30,  11,  21, 336]])